installing required packages

In [ ]:
! pip install requests

In [1]:
import requests
import json

generating embedding from Azure OpenAI Rest API

In [2]:
# Replace these placeholders with your actual values
aoai_api_key = "YOUR_API_KEY"
aoai_api_base_url = "https://YOUR_AZURE_OPENAI_URL.openai.azure.com/"
aoai_embedding_deployment_name = "YOUR_EMBEDDING_DEPLOYMENT_NAME"
aoai_api_version = "2024-02-01"


# Define the URL of the API endpoint
url = f"{aoai_api_base_url}openai/deployments/{aoai_embedding_deployment_name}/embeddings?api-version={aoai_api_version}"

# Define the headers, including the API key
headers = {
    'Content-Type': 'application/json',
    'api-key': aoai_api_key
}

# Define the JSON body of the request
body = {
    "input": "Who is walter white?"
}

# Convert the payload to a JSON string
str_body = json.dumps(body)

# Make the POST request with the headers and JSON body
response = requests.post(url, headers=headers, data=str_body)

embedding = None
# Check the response
if response.status_code == 200:
    print('Request successful!')
    json_response = response.json()
    embedding = json_response["data"][0]["embedding"]
else:
    print('Request failed!')
    print(response.text)


print(embedding)

ConnectionError: HTTPSConnectionPool(host='your_azure_openai_url.openai.azure.com', port=443): Max retries exceeded with url: /openai/deployments/YOUR_EMBEDDING_DEPLOYMENT_NAME/embeddings?api-version=2024-02-01 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000011AF0D89990>: Failed to resolve 'your_azure_openai_url.openai.azure.com' ([Errno 11001] getaddrinfo failed)"))

Calling Azure AI Search Service to do vector search

for this sample, we are calling a search index that has vectorized content of "Breaking Bad - 01 Pilot" screenplay

In [11]:
# Replace these placeholders with your actual values
aais_base_url = "https://YOUR_SEARCH_URL.search.windows.net/"
aais_index_name = "YOUR_INDEX_NAME"
aais_api_key = "YOUR_API_KEY"
aais_api_version = "2024-03-01-Preview"
aais_index_vector_field = "YOUR_VECTOR_FIELD_NAME"


# The URL for the request
url = f"{aais_base_url}/indexes/{aais_index_name}/docs/search?api-version={aais_api_version}"

# The headers for the request
headers = {
    "Content-Type": "application/json",
    "api-key": aais_api_key
}

# The body of the request
body = {
    "count": True,
    "select": "title, chunk", # other fields to return may go here
    "vectorQueries": [
        {
            "kind": "vector",
            "vector": embedding,
            "exhaustive": True,
            "fields":aais_index_vector_field,
            "k": 5
        }
    ]
}

# Make the POST request
response = requests.post(url, headers=headers, data=json.dumps(body))

# Check the response
if response.status_code == 200:
    print('Request successful!')
    json_response = response.json()
    json_str = json.dumps(json_response, indent=2)
    # Print the response
    print(json_str)
else:
    print('Request failed!')
    print(response.text)


Request successful!
{
  "@odata.context": "https://jhl-ai-search.search.windows.net/indexes('screenplay')/$metadata#docs(*)",
  "@odata.count": 5,
  "value": [
    {
      "@search.score": 0.8519684,
      "chunk": "... I\nhad all three of you in my heart.\n\nThe sirens are WAILING now, on top of us. WALTER WHITE, the \nunderpants man, turns off the camcorder. He carefully sets \nit on a bare patch of ground by his feet. Next to it he \nsets his wallet, lying open where it can be seen.\n\nCLOSE ON the wallet -- a photo ID card is visible. Walt's \nsmiling face is on it. It identifies him as a teacher at \nJ.P. Wynne High School, Ontario Unified School District.\n\nWalt pulls the chrome pistol from the back of his \nwaistband, aiming it across the tall weeds. It glints hard \nin the sun.\n\nFlashing red LIGHT BARS speed into view, skimming the tops \nof the weeds. Heading straight for us.\n\n\n\n4.\n\nWalt stands tall in his underpants, not flinching. Off him, \nready to shoot the first